# Process text for entire dataframe
- Remove punctuation, parentheses and convert to lowercase

In [273]:
import nltk
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Shamelessly borrowed code from 1) https://www2.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html  (tf-idf) 2)https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730 (LDA)

In [312]:
punc1 = '!"”#$%&\'()*+,./:;<>?@[\\]^_`{|}~…' # want to replace with ''
punc2 = '-=' # want to replace with ' '

In [296]:
stopwords_comprehensive = stopwords.words('english') + ['uhm', 'umm', 'md', 'so', 'pt', 'oth', 'um', 'legend', 'hmm', 'ah', 'na', 'mm' 'hm']

In [297]:
def remove_parentheses(txt):
    txt = re.sub('\([^)]*\)\)','', txt) # remove double parentheses 
    txt = re.sub(r'\([^)]*\)', '', txt) # remove single parentheses 
    return txt

In [298]:
def remove_numerical(txt):
    txt = re.sub('[0-9]+', '', txt)
    return txt

In [299]:
def remove_punc(txt):
    for a in punc1:
        txt = txt.replace(a,"")
    for b in punc2:
        txt = txt.replace(b," ")
    return txt

In [300]:
def lowercase(txt):
    txt = txt.lower()
    return txt

In [301]:
df = pd.read_csv('all_transcripts.csv')

In [302]:
df.head()

,Unnamed: 0,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,0,A003,A003 LEGEND: MD2=Physician PT=Patient MD2: So ...,NaN,VA,U,NaN
1,1,A004,A004 LEGEND: MD2=Physician OTH=Study staff-set...,NaN,VA,U,NaN
2,2,A014,A014 LEGEND: MD2=Physician PT=Patient MD2: Tha...,NaN,VA,U,NaN
3,3,A015,A015_CLEAN LEGEND: MD2=Physician PT=Patient MD...,NaN,VA,U,NaN
4,4,A016,A016_CLEAN_LOUD LEGEND: MD2=Physician PT: Pati...,NaN,VA,U,NaN


In [303]:
# clean up each of the convo_1 texts with the processing functions written above
for index, row in df.iterrows():
    if pd.isnull(row['Convo_1']) == False:
        row['Convo_1'] = remove_parentheses(row['Convo_1'])
        row['Convo_1'] = remove_numerical(row['Convo_1'])
        row['Convo_1'] = remove_punc(row['Convo_1'])
        row['Convo_1'] = lowercase(row['Convo_1'])
        df.set_value(index,'Convo_1', row['Convo_1'])

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


# Tokenizing, removing stopwords, stemming 

## Experiment on one documet

In [304]:
df.head()

,Unnamed: 0,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,0,A003,a legend md physician pt patient md so thank y...,NaN,VA,U,NaN
1,1,A004,a legend md physician oth study staff setting ...,NaN,VA,U,NaN
2,2,A014,a legend md physician pt patient md thank you ...,NaN,VA,U,NaN
3,3,A015,aclean legend md physician pt patient md so it...,NaN,VA,U,NaN
4,4,A016,acleanloud legend md physician pt patient md o...,NaN,VA,U,NaN


In [305]:
convo_0 = df['Convo_1'][0]

In [253]:
tokens = nltk.word_tokenize(convo_0)
tokens

['a',
 'legend',
 'md',
 'physician',
 'pt',
 'patient',
 'md',
 'so',
 'thank',
 'you',
 'for',
 'coming',
 'in',
 'i',
 'know',
 'we',
 'kinda',
 'moved',
 'your',
 'appointment',
 'upbut',
 'next',
 'week',
 'our',
 'clinic',
 'was',
 'kind',
 'of',
 'overbooked',
 'and',
 'we',
 'didn',
 't',
 'have',
 'staff',
 'to',
 'bring',
 'people',
 'in',
 'dr',
 'pt',
 'okay',
 'md',
 'um',
 'so',
 'ah',
 'we',
 'wanted',
 'to',
 'bring',
 'you',
 'in',
 'to',
 'talk',
 'about',
 'your',
 'biopsy',
 'results',
 'um',
 'as',
 'we',
 'kind',
 'of',
 'expected',
 'with',
 'your',
 'psa',
 'being',
 'up',
 'there',
 'was',
 'prostate',
 'cancer',
 'in',
 'the',
 'sample',
 'pt',
 'okay',
 'md',
 'um',
 'the',
 'ah',
 'you',
 'know',
 'the',
 'exam',
 'that',
 'they',
 'did',
 'they',
 'could',
 'feel',
 'a',
 'little',
 'area',
 'that',
 'felt',
 'abnormal',
 'on',
 'the',
 'left',
 'side',
 'of',
 'your',
 'prostate',
 'gland',
 'and',
 'that',
 's',
 'where',
 'the',
 'biopsies',
 'were',
 'p

In [254]:
filtered = [w for w in tokens if not w in stopwords_comprehensive]
filtered

['physician',
 'patient',
 'thank',
 'coming',
 'know',
 'kinda',
 'moved',
 'appointment',
 'upbut',
 'next',
 'week',
 'clinic',
 'kind',
 'overbooked',
 'staff',
 'bring',
 'people',
 'dr',
 'okay',
 'ah',
 'wanted',
 'bring',
 'talk',
 'biopsy',
 'results',
 'kind',
 'expected',
 'psa',
 'prostate',
 'cancer',
 'sample',
 'okay',
 'ah',
 'know',
 'exam',
 'could',
 'feel',
 'little',
 'area',
 'felt',
 'abnormal',
 'left',
 'side',
 'prostate',
 'gland',
 'biopsies',
 'positive',
 'nd',
 'uhh',
 'point',
 'gon',
 'na',
 'talk',
 'kind',
 'grade',
 'prostate',
 'cancer',
 'means',
 'kind',
 'treatment',
 'options',
 'know',
 'probably',
 'given',
 'instruction',
 'least',
 'handouts',
 'kinda',
 'go',
 'know',
 'treat',
 'prostate',
 'cancer',
 'she—if',
 'yet—she',
 'give',
 'educational',
 'today',
 'general',
 'grade',
 'prostate',
 'cancer',
 'looks',
 'microscope',
 'lump',
 'prostate',
 'cancers',
 'risk',
 'categories',
 'call',
 'either',
 'low',
 'risk',
 'intermediate',
 '

In [255]:
# check most common words see if there is anything we should add to stopwords list
count = Counter(filtered)
print(count.most_common(100))

[('cancer', 17), ('risk', 15), ('prostate', 13), ('know', 11), ('would', 9), ('four', 8), ('psa', 7), ('five', 7), ('things', 7), ('spread', 7), ('na', 6), ('treatment', 6), ('low', 6), ('ten', 6), ('like', 6), ('radiation', 6), ('kind', 5), ('okay', 5), ('microscope', 5), ('intermediate', 5), ('aggressive', 5), ('really', 5), ('plus', 5), ('enough', 5), ('active', 5), ('people', 4), ('talk', 4), ('biopsy', 4), ('could', 4), ('point', 4), ('gon', 4), ('grade', 4), ('options', 4), ('common', 4), ('even', 4), ('normal', 4), ('surveillance', 4), ('proton', 4), ('kinda', 3), ('ah', 3), ('little', 3), ('gland', 3), ('go', 3), ('looks', 3), ('high', 3), ('look', 3), ('terms', 3), ('see', 3), ('gleason', 3), ('six', 3), ('one', 3), ('three', 3), ('yeah', 3), ('start', 3), ('best', 3), ('older', 3), ('think', 3), ('worse', 3), ('core', 3), ('va', 3), ('therapy', 3), ('patient', 2), ('moved', 2), ('bring', 2), ('dr', 2), ('wanted', 2), ('results', 2), ('abnormal', 2), ('biopsies', 2), ('means',

In [256]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [257]:
stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)
count = Counter(stemmed)
print(count.most_common(100))


[('cancer', 19), ('risk', 15), ('prostat', 13), ('know', 11), ('psa', 9), ('would', 9), ('four', 8), ('thing', 8), ('look', 7), ('five', 7), ('spread', 7), ('biopsi', 6), ('na', 6), ('grade', 6), ('treatment', 6), ('low', 6), ('ten', 6), ('like', 6), ('radiat', 6), ('kind', 5), ('okay', 5), ('talk', 5), ('option', 5), ('microscop', 5), ('intermedi', 5), ('appear', 5), ('aggress', 5), ('realli', 5), ('plu', 5), ('normal', 5), ('start', 5), ('enough', 5), ('activ', 5), ('core', 5), ('peopl', 4), ('could', 4), ('point', 4), ('gon', 4), ('go', 4), ('categori', 4), ('term', 4), ('common', 4), ('even', 4), ('year', 4), ('surveil', 4), ('proton', 4), ('kinda', 3), ('ah', 3), ('littl', 3), ('gland', 3), ('treat', 3), ('call', 3), ('high', 3), ('see', 3), ('gleason', 3), ('six', 3), ('one', 3), ('three', 3), ('yeah', 3), ('get', 3), ('typic', 3), ('best', 3), ('older', 3), ('think', 3), ('wors', 3), ('va', 3), ('therapi', 3), ('patient', 2), ('come', 2), ('move', 2), ('bring', 2), ('dr', 2), ('

## Apply to process entire conversation 1 column

In [306]:
df_nona = df.dropna(subset=['Convo_1'])
conversations = df_nona['Convo_1']

In [259]:
stemmer = PorterStemmer()
stopwords_comprehensive = stopwords.words('english') + ['uhm', 'md3', 'md2', 'md', 'so', 'pt', 'oth', 'um', 'legend']

In [278]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    # filtered = [w for w in tokens if not w in stopwords_comprehensive]
    stems = stem_tokens(tokens, stemmer)
    return stems

In [261]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [280]:
tfidf = TfidfVectorizer(tokenizer=tokenize, max_df = 0.7, min_df = 3, max_features=5000, stop_words = stopwords_comprehensive)
tfs = tfidf.fit_transform(conversations)

In [281]:
features = tfidf.get_feature_names()

In [282]:
features

['aaah',
 'aah',
 'ab',
 'abat',
 'abdomen',
 'abdomin',
 'abil',
 'ablat',
 'abnorm',
 'about…',
 'about…i',
 'about…y',
 'abov',
 'abrupt',
 'abscess',
 'absenc',
 'absolut',
 'absorb',
 'abus',
 'abut',
 'ac',
 'academ',
 'acceler',
 'accent',
 'accept',
 'access',
 'accid',
 'accommod',
 'accomplish',
 'accord',
 'account',
 'accru',
 'accumul',
 'accur',
 'accuraci',
 'ach',
 'achi',
 'achiev',
 'achil',
 'acid',
 'aclean',
 'acleanloud',
 'acquaint',
 'across',
 'act',
 'action',
 'actor',
 'actually…',
 'acut',
 'ad',
 'adam',
 'adapt',
 'add',
 'addit',
 'address',
 'adenocarcinoma',
 'adenoid',
 'adequ',
 'adher',
 'adjac',
 'adjunct',
 'adjust',
 'adjuv',
 'administ',
 'administr',
 'admir',
 'admit',
 'admittedli',
 'adopt',
 'adult',
 'advair',
 'advanc',
 'advantag',
 'advers',
 'advertis',
 'advic',
 'advil',
 'advis',
 'advoc',
 'affect',
 'affili',
 'afford',
 'afraid',
 'african',
 'afternoon',
 'afterward',
 'afterword',
 'after…',
 'after…aft',
 'again…',
 'agent',
 

In [283]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    row = np.squeeze(Xtr[row_id].toarray())
    topn_ids = np.argsort(row)[::-1][:top_n] # get indices of biggest tf-idf coefficients 
    top_feats = [(features[i], row[i]) for i in topn_ids] # get corresponding feature/value tuple based off indices
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [284]:
top_feats_in_doc(tfs, features, 2, top_n=25)

,feature,tfidf
0,six,0.304083
1,spinal,0.241927
2,sphincter,0.233886
3,ultrasound,0.197440
4,rod,0.175926
5,flow,0.174610
6,push,0.154706
7,basic,0.130269
8,sling,0.129373
9,hold,0.127659


TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
IDF(t) = log(Total number of documents / Number of documents with term t in it).
To compute, multiply TF(t)*IDF(t) for each term. We get higher tf-idf scores for terms that are really important to certain documents (both in that they appear in few other documents, and appear a lot in a single document). 

# LDA

In [307]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
c_vectorizer= CountVectorizer(tokenizer=tokenize, max_df = 0.7, min_df = 3, max_features=5000, stop_words = stopwords_comprehensive)
cv = c_vectorizer.fit_transform(conversations)
cv_feature_names = c_vectorizer.get_feature_names()

In [308]:
no_topics = 20

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(cv)

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [309]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [310]:
no_top_words = 10
display_topics(lda, cv_feature_names, no_top_words)

Topic 0:
three seven six five yep “ va set leak posit
Topic 1:
typic robot six va set percent left took far five
Topic 2:
mm gon dr nerv three hi six robot core area
Topic 3:
typic robot six intermedi three incis five set basic nerv
Topic 4:
six three five four “ sort ten percent seven leakag
Topic 5:
three six “ five yep core ten seven percent four
Topic 6:
five percent six three ten “ four sort seven leak
Topic 7:
six three “ seven grade va yep ten leakag issu
Topic 8:
six percent “ three five benefit ten far incontin area
Topic 9:
inflat alrighti absorb mediocr melt aliv mister dozen vigor depend
Topic 10:
hmmm “ six ten percent seven three five benefit basic
Topic 11:
“ three six leakag five inform far took ten news
Topic 12:
six three ten “ percent yep five seven area eight
Topic 13:
six ten percent “ three seven grade five basic four
Topic 14:
six percent three ten five oper “ seven far sort
Topic 15:
three six seven nerv exactli “ basic yep far area
Topic 16:
three six “ five is